In [2]:
import time, numpy as np, tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)


In [3]:
max_features = 10_000      # vocab size used in class
max_len = 500              # pad/truncate length used in class

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test  = sequence.pad_sequences(x_test,  maxlen=max_len)

print("Train:", x_train.shape, "Test:", x_test.shape)


Train: (25000, 500) Test: (25000, 500)


In [4]:
model = Sequential([
    layers.Embedding(max_features, 128, input_length=max_len),
    layers.SimpleRNN(128, activation="tanh"),   # stable choice for RNNs
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Force weight initialization so summary is fully populated
model.build(input_shape=(None, max_len))
model.summary()   # <-- paste this into your Word report


C:\Users\sande\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
earlystopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

start = time.time()
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[earlystopping],
    verbose=1
)
elapsed = time.time() - start
print(f"Total training time: {elapsed:.2f} sec")  


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - accuracy: 0.5598 - loss: 0.6762 - val_accuracy: 0.6054 - val_loss: 0.6437
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.6671 - loss: 0.6030 - val_accuracy: 0.6282 - val_loss: 0.6370
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.7204 - loss: 0.5403 - val_accuracy: 0.6634 - val_loss: 0.6406
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.7714 - loss: 0.4768 - val_accuracy: 0.6904 - val_loss: 0.6301
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.7913 - loss: 0.4485 - val_accuracy: 0.6998 - val_loss: 0.6808
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.8443 - loss: 0.3711 - val_accuracy: 0.7212 - val_loss: 0.6411
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.8691 - loss: 0.3259 - val_accuracy: 0.7304 - val_loss: 0.6480
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.8225 - loss: 0.3800 - 

In [8]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print("Test accuracy:", f"{test_acc:.4f}")


Test accuracy: 0.6865


In [9]:
model.save("simple_rnn_imdb.keras")
